In [1]:
from nltk.corpus import brown
from nltk.corpus import stopwords
import re
from collections import Counter
import math
import random

In [2]:
stop_words = set(stopwords.words('english'))

In [3]:
data = list(brown.words())

In [4]:
len(data)

1161192

In [5]:
def process(word):
    w = word.lower()
    w = re.sub(r'[^\w\s]', '', w)
    if w in stop_words:
        return ''
    return w

In [6]:
ls = [process(i) for i in data]
ls = [i for i in ls if i!='']
len(ls)

539921

In [7]:
cnt = Counter(ls)
V = [i[0] for i in cnt.most_common(5000)]
C = [i[0] for i in cnt.most_common(1000)]

In [8]:
N = len(ls)
prcw = [[0]*len(C) for i in range(len(V))]
for i in range(N):
    if not ls[i] in V: continue
    row = V.index(ls[i])
        
    if(i-2>=0 and ls[i-2] in C):
        col = C.index(ls[i-2])
        prcw[row][col] += 1
    
    if(i-1>=0 and ls[i-1] in C):
        col = C.index(ls[i-1])
        prcw[row][col] += 1
        
    if(i+1<N and ls[i+1] in C):
        col = C.index(ls[i+1])
        prcw[row][col] += 1
        
    if(i+2<N and ls[i+2] in C):
        col = C.index(ls[i+2])
        prcw[row][col] += 1
    

In [9]:
for i in range(len(prcw)):
    denom = sum(prcw[i])
    prcw[i] = [j/denom for j in prcw[i]]

In [10]:
# for i in range(len(prcw)):
#     for j in range(len(prcw[0])):
#         if(V[i]==C[j]):
#             prcw[i][j]=1

In [11]:
pc = [cnt[i]/N for i in C]
pw = [cnt[i]/N for i in V]

In [12]:
mutual_inf = []
for i in range(len(prcw)):
    temp = []
    for j in range(len(prcw[0])):
        if(prcw[i][j]==0):
            mu = 0
        else:
            mu = max(0,math.log(prcw[i][j]/pc[j]))
        temp.append(mu)
    mutual_inf.append(temp)   

In [13]:
PMI_matrix = mutual_inf

In [16]:
from sklearn.decomposition import PCA
import numpy as np

row_means = np.mean(PMI_matrix, axis=1, keepdims=True)
col_means = np.mean(PMI_matrix, axis=0, keepdims=True)
centered_PMI = PMI_matrix - row_means - col_means + np.mean(PMI_matrix)

num_dimensions = 100  # Adjust as needed
pca = PCA(n_components=num_dimensions)
lower_dim_embedding = pca.fit_transform(centered_PMI)

In [17]:
lower_dim_embedding.shape

(5000, 100)

In [18]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=2)
neigh.fit(lower_dim_embedding,[1 for _ in range(5000)])

KNeighborsClassifier(n_neighbors=2)

In [19]:
# samp = random.choices([i for i in range(len(V))],k=25)
# for s in samp:
#     nn = neigh.kneighbors([lower_dim_embedding[s]],return_distance = False)
#     #print(nn)
#     print("Closest match for " +V[s] +" is "+V[nn[0][1]])
    

In [20]:
from sklearn.cluster import KMeans
import numpy as np

kmeans = KMeans(n_clusters=100, random_state=0, n_init="auto").fit(lower_dim_embedding)

In [21]:
labels = kmeans.predict(lower_dim_embedding)

In [22]:
clusters = {}
n = 0
for item in labels:
    if item in clusters:
        clusters[item].append(V[n])
    else:
        clusters[item] = [V[n]]
    n +=1

In [29]:
item = 95
print("Cluster "+ str(item))
print(list(clusters[item]))

Cluster 95
['new', 'city', 'york', 'washington', 'island', 'central', 'st', 'county', 'club', 'district', 'reported', 'hospital', 'chicago', 'park', 'london', 'post', 'governor', 'bank', 'providence', 'editor', 'louis', 'texas', 'san', 'california', 'located', 'boston', 'dallas', 'co', 'los', 'issued', 'philadelphia', 'moscow', 'georgia', 'pennsylvania', 'orleans', 'manchester', 'tour', 'illinois', 'downtown', 'mayor', 'attended', 'atlanta', 'factory', 'dc', 'kansas', 'yankees', 'greenwich', 'neighboring', 'portland', 'memorial', 'maryland', 'madison', 'capitol', 'miami', 'port', 'detroit', 'citys', 'missouri', 'warwick']
